In [ ]:
import os
import av
import numpy as np
import cv2
import csv

grid_size = 4  # How many rows and columns we're splitting the frame into
bin_count = 12  # Number of color shades we're counting in each histogram

video_folder = 'Target_video/Sword'

def process_videos_in_folder(folder_path):
    parent_folder = os.path.abspath(os.path.join(folder_path, os.pardir))
    
    results_folder = os.path.join(parent_folder, "hello sample")  # Where the results are saved
    os.makedirs(results_folder, exist_ok=True)  # Create the folder if it's not there
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith((".avi")):  # We only want video files
                video_path = os.path.join(root, file)
                video_name = os.path.splitext(os.path.relpath(video_path, folder_path))[0]
                analyze_video(video_path, video_name, results_folder)

def extract_key_frames(video_path):
    try:
        video = av.open(video_path)
        all_frames = [frame for frame in video.decode(video=0)]
        key_frames = [all_frames[0], all_frames[len(all_frames) // 2], all_frames[-1]] # get 1st frame,middle frame, and the last frame
        return key_frames
    except UnicodeDecodeError as e:
        print(f"Oops! Can't read this file: {video_path}. Error: {e}")
        return None
    except Exception as e:
        print(f"Something went wrong with {video_path}. Error: {e}")
        return None

def analyze_video(video_path, video_name, results_folder):
    key_frames = extract_key_frames(video_path)
    if key_frames is None:
        print(f"Skipping {video_path}. Couldn't get the frames we needed.")  # User-friendly skip message
        return
    video_results_folder = os.path.join(results_folder, f'{video_name}_output')
    os.makedirs(video_results_folder, exist_ok=True)
    for frame_number, frame in enumerate(key_frames, 1):
        create_color_histogram(frame, grid_size, bin_count, frame_number, video_results_folder)

def create_color_histogram(frame, grid_size, bin_count, frame_number, output_folder):
    try:
        image = np.array(frame.to_image())  # Convert frame to image
    except Exception as e:
        print(f"Couldn't convert frame to image. Skipping.")
        return None
    
    height, width, _ = image.shape
    cell_height, cell_width = height // grid_size, width // grid_size
    
    all_histograms = []
    for row in range(grid_size):
        for col in range(grid_size):
            cell = image[row * cell_height:(row + 1) * cell_height, 
                         col * cell_width:(col + 1) * cell_width]
            red_hist = cv2.calcHist([cell], [0], None, [bin_count], [0, 256]).flatten()
            green_hist = cv2.calcHist([cell], [1], None, [bin_count], [0, 256]).flatten() # Count the colors in each grid block
            blue_hist = cv2.calcHist([cell], [2], None, [bin_count], [0, 256]).flatten()
            
            red_hist /= red_hist.sum()
            green_hist /= green_hist.sum()  #color counts add up to 100%
            blue_hist /= blue_hist.sum()
            
            combined_histogram = [sum(x) for x in zip(red_hist, green_hist, blue_hist)]# Combine the color counts
            all_histograms.append(combined_histogram)
    
    # Save our color counts to a file
    csv_filename = os.path.join(output_folder, f'{output_folder}_histogram_frame_{frame_number}.csv')
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f'Bin {i+1}' for i in range(bin_count)])  # Labels for the columns
        for hist in all_histograms:
            writer.writerow(hist)
    
    print(f"Saved our color data to {csv_filename}")
    
process_videos_in_folder(video_folder)
video_folder = 'Target_video/Sword_exercise'
process_videos_in_folder(video_folder)
video_folder = 'Target_video/Wave'
process_videos_in_folder(video_folder)
video_folder = 'Target_video/Ride_bike'
process_videos_in_folder(video_folder)
video_folder = 'Target_video/Drink'
process_videos_in_folder(video_folder)
video_folder = 'Target_video/Cartwheel'
process_videos_in_folder(video_folder)